# Your Details

Your Name: Siddartha Sandeep Peddada

Your ID Number: 24192929

# Etivity Part 2: Quantizing a TensorFlow/Keras Model


* Understand Quantizations in TensorFlow
* Quantize a CNN using the TensorFlow Model optimisation framework
* Analyse the model perfromance
* Results analysis

### Let's get started!

    [1] Import data dependencies
    [2] Generate a TensorFlow/keras CNN model for the Fashion MNIST dataset
    [3] Convert model to TF Lite model
    [4] Perform Post Training Quantization (PTQ) to generate TF Lite model for:
        (a) PTQ using Float 16 Quantization
        (b) PTQ using Dynamic Range Quantization
        (c) PTQ using Full Integer (int8) Quantization
        (d) Evaluate the TF Lite models
    [5] Perform Quantization Aware Training (QAT)
        (a) Train a TF model through tf.keras
        (b) Make it quantization-aware
        (c) Quantize the model using Dynamic Range Quantization
        (d) Evaluate the TF Lite model performance
    

### Installing the TensorFlow Model Optimisation toolkit

You must first install it using pip (comment this out once you have done this).


In [ ]:
# Install the TF optimization toolkit the first time


#!pip install -q tensorflow-model-optimization


In [ ]:
import tensorflow as tf
print(tf.__version__)


2.19.0


## 1. Import the data dependencies

In [ ]:
import numpy as np
import tensorflow as tf
import time
import os
import pathlib
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from tensorflow import keras

In [ ]:
# Check that we are using a GPU
physical_devices = tf.config.experimental.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

Num GPUs Available:  0


In [ ]:
import os
import tensorflow as tf
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
print("Available devices:", tf.config.list_physical_devices())


Available devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]


## 2. Generate a TensorFlow Model

We'll build a CNN model to classify the 10 fashion item categories from the [FASHION_MNIST dataset](https://www.tensorflow.org/datasets/catalog/fashion_mnist).

This training won't take long because you're training the model for just 5 epochs, which trains to about ~90% accuracy.

In [ ]:
# Load Fashion MNIST dataset
fashion_mnist = tf.keras.datasets.fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# Reshape data for CNN input
img_width, img_height = 28, 28
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)
input_shape = (img_width, img_height, 1)

# Normalize the input image so that each pixel value is between 0 to 1.
X_train = X_train.astype(np.float32) / 255.0
X_test = X_test.astype(np.float32) / 255.0


# Define the model architecture
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.1), # Randomly disable 10% of neurons
    tf.keras.layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
    tf.keras.layers.Dropout(rate=0.1), # Randomly disable 10% of neurons
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(100, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])


# Build the model
model.compile(
    loss=tf.keras.losses.sparse_categorical_crossentropy, # loss function
    optimizer=tf.keras.optimizers.Adam(), # optimizer function
    metrics=['accuracy'] # reporting metric
)


# Train the fashion MNIST classification model
with tf.device('/CPU:0'):
    model.fit(
      X_train,
      y_train,
      epochs=5,
      validation_split=0.1,
    )

/opt/anaconda3/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 21s 12ms/step - accuracy: 0.7557 - loss: 0.6559 - val_accuracy: 0.8740 - val_loss: 0.3353
Epoch 2/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 17s 10ms/step - accuracy: 0.8784 - loss: 0.3337 - val_accuracy: 0.8862 - val_loss: 0.3068
Epoch 3/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 16s 9ms/step - accuracy: 0.8981 - loss: 0.2769 - val_accuracy: 0.9042 - val_loss: 0.2615
Epoch 4/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.9081 - loss: 0.2444 - val_accuracy: 0.8892 - val_loss: 0.2953
Epoch 5/5
1688/1688 ━━━━━━━━━━━━━━━━━━━━ 18s 11ms/step - accuracy: 0.9182 - loss: 0.2184 - val_accuracy: 0.9087 - val_loss: 0.2453


**Evaluate and save the model**

In [ ]:
score = model.evaluate(X_test, y_test, verbose=1)
print("Test loss {:.4f}, accuracy {:.2f}%".format(score[0], score[1] * 100))

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.9049 - loss: 0.2598
Test loss 0.2534, accuracy 90.63%


In [ ]:
model.save("models/new_model.h5")
print("Saved model to disk")

Saved model to disk


## 3. Convert the trained model to TensorFlow Lite format

In the code cell below, convert the model to a **TensorFlow Lite** model and then save this unquantized TFLite model to the ./fashion_mnist_tflite_model directory

In [ ]:
model = tf.keras.models.load_model('models/new_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpfxhvb4lp/assets


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpfxhvb4lp/assets


Saved artifact at '/var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpfxhvb4lp'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  13161036112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13399333904: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483812688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483813840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483812496: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483807120: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483807696: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483806160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483806352: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483805968: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742779117.654095 8917793 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742779117.654722 8917793 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 01:18:37.656274: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpfxhvb4lp
2025-03-24 01:18:37.656707: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 01:18:37.656712: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpfxhvb4lp
2025-03-24 01:18:37.664170: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 01:18:37.725301: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpfxhvb4lp
2025-03-24 01:18:37.731669: I tensorflow/cc/saved_model/loader.cc:

It's now a TensorFlow Lite model, but it's still using 32-bit float values for all parameter data.

In [ ]:
import pathlib
tflite_models_dir = pathlib.Path("./fashion_mnist_tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

# Save the unquantized float model:
tflite_model_file = tflite_models_dir/"fashion_mnist_model.tflite"
tflite_model_file.write_bytes(tflite_model)

1825276

## 4. Post-Training Quantization (PTQ)

### Part (a): PTQ using Float 16 Quantization
Here for post-training float 16 quantization and then evaluate the file size compared to the unquantized tflite model size.

In [ ]:
# Convert using Float16 Quantization
model = tf.keras.models.load_model('models/new_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
tflite_quant16_model = converter.convert()

# Save the quantized 16-bit model:
tflite_quant16_model_file = tflite_models_dir/"fashion_model_quant16.tflite"
tflite_quant16_model_file.write_bytes(tflite_quant16_model)


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpl16dq_ph/assets


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpl16dq_ph/assets


Saved artifact at '/var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpl16dq_ph'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  13483818256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275183888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275184656: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275182544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275184464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275181392: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275181968: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275179856: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275180432: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5275185232: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742779230.454108 8917793 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742779230.454138 8917793 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 01:20:30.454308: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpl16dq_ph
2025-03-24 01:20:30.454748: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 01:20:30.454752: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpl16dq_ph
2025-03-24 01:20:30.463695: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 01:20:30.487852: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpl16dq_ph
2025-03-24 01:20:30.494378: I tensorflow/cc/saved_model/loader.cc:

915704

**Evaluate the reduction in size of the model** - how much smaller is the Quantized 16-bit model?

In [ ]:

print("Float model in Mb:", os.path.getsize(tflite_model_file) / float(2**20))
print("Quantized 16-bit model in Mb:", os.path.getsize(tflite_quant16_model_file) / float(2**20))
print("Compression ratio:", os.path.getsize(tflite_model_file)/os.path.getsize(tflite_quant16_model_file))


Float model in Mb: 1.7407188415527344
Quantized 16-bit model in Mb: 0.8732833862304688
Compression ratio: 1.9933035129255743


In [ ]:
compression_ratio = os.path.getsize(tflite_model_file) / os.path.getsize(tflite_quant16_model_file)
print(f"Compression ratio: {compression_ratio:}")

Compression ratio: 1.9933035129255743


### Part (b): PTQ using Dynamic Range Quantization
Next quantize the original model dynamically to change the model weight and activations from float to int8 format. Convert the model using **Dynamic Range Quantization** and evaluate the model file size reduction.

In [ ]:

model = tf.keras.models.load_model('models/new_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

# Save the quantized model:
tflite_quant_model_file = tflite_models_dir/"fashion_model_quant.tflite"
tflite_quant_model_file.write_bytes(tflite_quant_model)


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpyo6g2f_f/assets


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpyo6g2f_f/assets


Saved artifact at '/var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpyo6g2f_f'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  13483806544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483812304: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483806160: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483808080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483808272: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483807888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483807504: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483803280: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483803472: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13483804816: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742779424.176053 8917793 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742779424.176329 8917793 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 01:23:44.177692: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpyo6g2f_f
2025-03-24 01:23:44.178346: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 01:23:44.178351: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpyo6g2f_f
2025-03-24 01:23:44.184559: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 01:23:44.249420: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpyo6g2f_f
2025-03-24 01:23:44.255311: I tensorflow/cc/saved_model/loader.cc:

469432

 **Evaluate the reduction in size of the model** - how much smaller is the Quantized model?

In [ ]:
print("Float model in Mb:", os.path.getsize(tflite_model_file) / float(2**20))
print("Quantized model in Mb:", os.path.getsize(tflite_quant_model_file) / float(2**20))
print("Compression ratio:", os.path.getsize(tflite_model_file)/os.path.getsize(tflite_quant_model_file))

Float model in Mb: 1.7407188415527344
Quantized model in Mb: 0.44768524169921875
Compression ratio: 3.8882649670239777


In [ ]:
# Load TFLite model and allocate tensors.
interpreter = \
tf.lite.Interpreter(model_path=str(tflite_quant_model_file))
interpreter.allocate_tensors()
# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
# Test model on some input data.
input_shape = input_details[0]['shape']
acc=0
for i in range(len(X_test)):
    input_data = X_test[i].reshape(input_shape)
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
#    if(np.argmax(output_data) == np.argmax(test_labels[i])):
    if(np.argmax(output_data) == y_test[i]):
        acc+=1
acc = acc/len(X_test)
print(acc*100)

90.64


### Part (c): PTQ using Full Integer (int8) Quantization
Convert the original model to satisfy **full integer quantization** so that everything is converted (including activations) from float32 into int8 format. Evaluate the model file size reduction. Note you will need to use the OPTIMIZE_FOR_SIZE option by using a small representative dataset of the model and also make sure the input and output tensors are in int8 format.

**Check that the input and output tensors are in int8 format**

In [ ]:
import tensorflow as tf

# Load and preprocess Fashion MNIST data
fashion_mnist_train, _ = tf.keras.datasets.fashion_mnist.load_data()
images = tf.cast(fashion_mnist_train[0], tf.float32) / 255.0  # Normalize to [0, 1]
images = tf.expand_dims(images, axis=-1)  # Add channel dimension: (28, 28, 1)

# Create dataset with batch size 1
mnist_ds = tf.data.Dataset.from_tensor_slices((images)).batch(1)

# Define representative dataset generator for calibration
def representative_data_gen():
    for input_value in mnist_ds.take(100):  # Use 100 samples for calibration
        yield [tf.cast(input_value, tf.float32)]

# Load the Keras model
model = tf.keras.models.load_model('models/new_model.h5')

# Create TFLite converter
converter = tf.lite.TFLiteConverter.from_keras_model(model)

# Use updated optimization setting
converter.optimizations = [tf.lite.Optimize.DEFAULT]

# Assign the representative dataset
converter.representative_dataset = representative_data_gen

# Force full integer quantization (including inputs/outputs)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8

# Convert the model
tflite_fullquant_model = converter.convert()

# Save the quantized model
with open('models/new_model_full_integer_quant.tflite', 'wb') as f:
    f.write(tflite_fullquant_model)

# Verify input/output types
interpreter = tf.lite.Interpreter(model_content=tflite_fullquant_model)
input_type = interpreter.get_input_details()[0]['dtype']
output_type = interpreter.get_output_details()[0]['dtype']
print('Input tensor type:', input_type)
print('Output tensor type:', output_type)


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpou_oz0db/assets


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpou_oz0db/assets


Saved artifact at '/var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpou_oz0db'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  13399334672: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13399330832: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323852112: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323853264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323851920: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323854416: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323853840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323855376: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323855184: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5323856144: TensorSpec(shape=(), dtype=tf.resource, name=None)


/opt/anaconda3/lib/python3.12/site-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1742779992.964734 8917793 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742779992.964773 8917793 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 01:33:12.964994: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpou_oz0db
2025-03-24 01:33:12.965870: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 01:33:12.965877: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpou_oz0db
2025-03-24 01:33:12.974815: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 01:33:13.005589: I tensorflow/cc/saved_model/

Input tensor type: <class 'numpy.int8'>
Output tensor type: <class 'numpy.int8'>


6, input_inference_type: INT8, output_inference_type: INT8
/opt/anaconda3/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
def representative_data_gen():
  for input_value in tf.data.Dataset.from_tensor_slices(X_train).batch(1).take(100):
    yield [input_value]

model = tf.keras.models.load_model('models/new_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_data_gen
# Ensure that if any ops can't be quantized, the converter throws an error
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
# Set the input and output tensors to uint8 (APIs added in r2.3)
converter.inference_input_type = tf.uint8
converter.inference_output_type = tf.uint8

tflite_fullquant_model = converter.convert()

# Saving the fully-quantized 8-bit model:
tflite_fullquant_model_file = tflite_models_dir/"mnist_model_fullquant.tflite"
tflite_fullquant_model_file.write_bytes(tflite_fullquant_model)

INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpxs4nm5tk/assets


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpxs4nm5tk/assets


Saved artifact at '/var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpxs4nm5tk'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  5235688464: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235691344: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235689808: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235692688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235690768: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235693840: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235693264: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235694800: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235694608: TensorSpec(shape=(), dtype=tf.resource, name=None)
  5235695568: TensorSpec(shape=(), dtype=tf.resource, name=None)


/opt/anaconda3/lib/python3.12/site-packages/tensorflow/lite/python/convert.py:854: UserWarning: Statistics for quantized inputs were expected, but not specified; continuing anyway.
  warnings.warn(
W0000 00:00:1742780438.048031 8917793 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742780438.049056 8917793 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 01:40:38.050102: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpxs4nm5tk
2025-03-24 01:40:38.050557: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 01:40:38.050561: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmpxs4nm5tk
2025-03-24 01:40:38.059620: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 01:40:38.128600: I tensorflow/cc/saved_model/

472568

 **Evaluate the reduction in size of the model** - how much smaller is the Quantized model?

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_fullquant_model)
input_type = interpreter.get_input_details()[0]['dtype']
print('input: ', input_type)
output_type = interpreter.get_output_details()[0]['dtype']
print('output: ', output_type)

input:  <class 'numpy.uint8'>
output:  <class 'numpy.uint8'>


/opt/anaconda3/lib/python3.12/site-packages/tensorflow/lite/python/interpreter.py:457: UserWarning:     Warning: tf.lite.Interpreter is deprecated and is scheduled for deletion in
    TF 2.20. Please use the LiteRT interpreter from the ai_edge_litert package.
    See the [migration guide](https://ai.google.dev/edge/litert/migration)
    for details.
    
  warnings.warn(_INTERPRETER_DELETION_WARNING)


In [ ]:
print("Float model in Mb:", os.path.getsize(tflite_model_file) / float(2**20))
print("Full Integer Quantized model in Mb:", os.path.getsize(tflite_fullquant_model_file) / float(2**20))
print("Compression ratio:", os.path.getsize(tflite_model_file)/os.path.getsize(tflite_fullquant_model_file))

Float model in Mb: 1.7407188415527344
Full Integer Quantized model in Mb: 0.45067596435546875
Compression ratio: 3.862462121853363


### Part (d):  Evaluate the TF Lite models on all images

In this section, evaluate the four TF Lite models by running inference using the TensorFlow Lite [`Interpreter`](https://www.tensorflow.org/api_docs/python/tf/lite/Interpreter) to compare the model accuracies. First, build a **run_tflite_model()** function to run inference on a TF Lite model and then an **evaluate_model()** function to evaluate the TF Lite model on all images in the X_test dataset.

**Evaluate the model performance for these models** by reporting on the model accuracies.
1. Float model (Unquantized)
2. 16-bit quantized model
3. Initial quantized 8-bit model
4. Fully quantized 8-bit model

In [ ]:
def run_tflite_model(tflite_file, test_image_indices):
  global test_images

  # Initialize the interpreter
  interpreter = tf.lite.Interpreter(model_path=str(tflite_file))
  interpreter.allocate_tensors()

  input_details = interpreter.get_input_details()[0]
  output_details = interpreter.get_output_details()[0]

  predictions = np.zeros((len(test_image_indices),), dtype=int)
  for i, test_image_index in enumerate(test_image_indices):
    test_image = X_test[test_image_index]
    test_label = y_test[test_image_index]

    # Check if the input type is quantized, then rescale input data to uint8
    if input_details['dtype'] == np.uint8:
      input_scale, input_zero_point = input_details["quantization"]
      test_image = test_image / input_scale + input_zero_point

    test_image = np.expand_dims(test_image, axis=0).astype(input_details["dtype"])
    interpreter.set_tensor(input_details["index"], test_image)
    interpreter.invoke()
    output = interpreter.get_tensor(output_details["index"])[0]

    predictions[i] = output.argmax()

  return predictions

In [ ]:
# Helper function to evaluate a TFLite model on all images
def evaluate_model(tflite_file, model_type):
  global test_images
  global test_labels

  test_image_indices = range(X_test.shape[0])
  predictions = run_tflite_model(tflite_file, test_image_indices)

  accuracy = (np.sum(y_test== predictions) * 100) / len(X_test)

  print('%s model accuracy is %.4f%% (Number of test samples=%d)' % (
      model_type, accuracy, len(X_test)))

1. Evaluate the float model

In [ ]:
evaluate_model(tflite_model_file, model_type="Float")

Float model accuracy is 90.6300% (Number of test samples=10000)


2. Evaluate the 16-bit quantized model

In [ ]:
evaluate_model(tflite_quant16_model_file, model_type="16-bit Quantized")

16-bit Quantized model accuracy is 90.6300% (Number of test samples=10000)


3. Evaluate the initial quantized 8-bit model

In [ ]:
evaluate_model(tflite_quant_model_file, model_type="Quantized")

Quantized model accuracy is 90.6400% (Number of test samples=10000)


4. Evaluate the fully quantized 8-bit integer model

In [ ]:
evaluate_model(tflite_fullquant_model_file, model_type="Fully Quantized")

Fully Quantized model accuracy is 90.5900% (Number of test samples=10000)


## 5. Quantization-Aware Training (QAT)

QAT models quantization during training and typically provides higher accuracies as compared to post-training quantization.
Generally, QAT is a three-step process:

    (a) Train a regular model through tf.keras
        YOU MAY HAVE TO 'import tf_keras as keras' and use model = keras.Sequential([...]) format.
    (b) Make it quantization-aware by applying the related API, allowing it to learn those loss-robust parameters.
    (c) Quantize the model use one of the approaches mentioned above and analyse performance


### **Part (a)**: Train a model for the FASHION MNIST dataset again

In [ ]:
# Load and preprocess Fashion MNIST dataset
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# Add channel dimension: (28, 28, 1)
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

# Define a simple CNN model
model = keras.Sequential([
    keras.layers.Input(shape=(28, 28, 1)),
    keras.layers.Conv2D(32, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=5, validation_split=0.1)


Epoch 1/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.3995 - accuracy: 0.8594 - val_loss: 0.3182 - val_accuracy: 0.8820
Epoch 2/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2707 - accuracy: 0.9025 - val_loss: 0.2743 - val_accuracy: 0.9017
Epoch 3/5
1688/1688 [==============================] - 9s 5ms/step - loss: 0.2253 - accuracy: 0.9168 - val_loss: 0.2522 - val_accuracy: 0.9103
Epoch 4/5
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1911 - accuracy: 0.9297 - val_loss: 0.2543 - val_accuracy: 0.9107
Epoch 5/5
1688/1688 [==============================] - 10s 6ms/step - loss: 0.1662 - accuracy: 0.9389 - val_loss: 0.2363 - val_accuracy: 0.9170


### Part (b): Make the model quantization aware
Hint: Use q_aware_model = quantize_model(model)

In [ ]:
!pip install keras-core
import tf_keras as keras
import tensorflow_model_optimization as tfmot

In [ ]:
quantize_model = tfmot.quantization.keras.quantize_model

# q_aware stands for for quantization aware.
q_aware_model = quantize_model(model)

# `quantize_model` requires a recompile.
q_aware_model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])

q_aware_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 quantize_layer (QuantizeLa  (None, 28, 28, 1)         3         
 yer)                                                            
                                                                 
 quant_conv2d (QuantizeWrap  (None, 26, 26, 32)        387       
 perV2)                                                          
                                                                 
 quant_max_pooling2d (Quant  (None, 13, 13, 32)        1         
 izeWrapperV2)                                                   
                                                                 
 quant_flatten (QuantizeWra  (None, 5408)              1         
 pperV2)                                                         
                                                                 
 quant_dense (QuantizeWrapp  (None, 128)               6

#### Retrain the quantization aware model

In [ ]:
q_aware_model.fit(
  x_train,
  y_train,
  epochs=5,
  validation_split=0.1
  #validation_data=(test_images, test_labels)
)

Epoch 1/5
1688/1688 [==============================] - 12s 7ms/step - loss: 0.1435 - accuracy: 0.9469 - val_loss: 0.2394 - val_accuracy: 0.9192
Epoch 2/5
1688/1688 [==============================] - 14s 8ms/step - loss: 0.1198 - accuracy: 0.9556 - val_loss: 0.2684 - val_accuracy: 0.9147
Epoch 3/5
1688/1688 [==============================] - 13s 8ms/step - loss: 0.1009 - accuracy: 0.9625 - val_loss: 0.2621 - val_accuracy: 0.9213
Epoch 4/5
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0846 - accuracy: 0.9692 - val_loss: 0.2736 - val_accuracy: 0.9177
Epoch 5/5
1688/1688 [==============================] - 12s 7ms/step - loss: 0.0724 - accuracy: 0.9740 - val_loss: 0.2870 - val_accuracy: 0.9228


#### Compare the accuracy of the baseline model to the new QAT model

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=1)

_, q_aware_model_accuracy = q_aware_model.evaluate(
    x_test, y_test, verbose=1)
print("\n-------------------------------------------------------------")
print('Baseline test accuracy:', baseline_model_accuracy*100)
print('Quant test accuracy:', q_aware_model_accuracy*100)

313/313 [==============================] - 1s 2ms/step - loss: 0.3167 - accuracy: 0.9167

-------------------------------------------------------------
Baseline test accuracy: 91.29999876022339
Quant test accuracy: 91.6700005531311


#### Fine tune with QAT on a subset of the training data

In [ ]:
q_aware_model.fit(
  x_train[:1000],
  y_train[:1000],
  epochs=5,
  validation_split=0.1
)

Epoch 1/5
29/29 [==============================] - 0s 16ms/step - loss: 0.0552 - accuracy: 0.9844 - val_loss: 0.0422 - val_accuracy: 0.9900
Epoch 2/5
29/29 [==============================] - 0s 8ms/step - loss: 0.0212 - accuracy: 0.9956 - val_loss: 0.0450 - val_accuracy: 0.9900
Epoch 3/5
29/29 [==============================] - 0s 12ms/step - loss: 0.0106 - accuracy: 1.0000 - val_loss: 0.0573 - val_accuracy: 0.9700
Epoch 4/5
29/29 [==============================] - 1s 17ms/step - loss: 0.0067 - accuracy: 1.0000 - val_loss: 0.0463 - val_accuracy: 0.9800
Epoch 5/5
29/29 [==============================] - 0s 11ms/step - loss: 0.0048 - accuracy: 1.0000 - val_loss: 0.0533 - val_accuracy: 0.9800


#### Re-evaluate the model accuracies.

In [ ]:
_, baseline_model_accuracy = model.evaluate(
    x_test, y_test, verbose=1)

_, q_aware_model_accuracy = q_aware_model.evaluate(
    x_test, y_test, verbose=1)
print("\n-------------------------------------------------------------")
print('Baseline test accuracy:', baseline_model_accuracy*100)
print('Quant test accuracy:', q_aware_model_accuracy*100)

313/313 [==============================] - 1s 2ms/step - loss: 0.3337 - accuracy: 0.9172

-------------------------------------------------------------
Baseline test accuracy: 91.29999876022339
Quant test accuracy: 91.72000288963318


#### Save the QAT model to the ./models directory

In [ ]:
#Save the entire model into a qat_model.h5 file
model.save("models/qat_model.h5")
print("Saved model to disk")

Saved model to disk


/opt/anaconda3/lib/python3.12/site-packages/tf_keras/src/engine/training.py:3098: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native TF-Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


### Part (c): Convert the model to TF Lite format  using Dynamic Range Quantization

In [ ]:
model = tf.keras.models.load_model('models/qat_model.h5')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantaware_model = converter.convert()

# Saving the quantized aware model:
tflite_quantaware_model_file = tflite_models_dir/"mnist_model_quantaware.tflite"
tflite_quantaware_model_file.write_bytes(tflite_quantaware_model)

/opt/anaconda3/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py:86: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmptc5zdifo/assets


INFO:tensorflow:Assets written to: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmptc5zdifo/assets


Saved artifact at '/var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmptc5zdifo'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 1), dtype=tf.float32, name='input_1')
Output Type:
  TensorSpec(shape=(None, 10), dtype=tf.float32, name=None)
Captures:
  5365444688: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6293380880: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6293388368: TensorSpec(shape=(), dtype=tf.resource, name=None)
  6293386256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13452606544: TensorSpec(shape=(), dtype=tf.resource, name=None)
  13452601552: TensorSpec(shape=(), dtype=tf.resource, name=None)


W0000 00:00:1742781763.862796 8917793 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1742781763.863045 8917793 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-03-24 02:02:43.864358: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmptc5zdifo
2025-03-24 02:02:43.865077: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-03-24 02:02:43.865082: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmptc5zdifo
2025-03-24 02:02:43.870760: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-03-24 02:02:43.965772: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /var/folders/d3/gzz38_ss2ylbt0sgg3k_jkym0000gn/T/tmptc5zdifo
2025-03-24 02:02:43.970689: I tensorflow/cc/saved_model/loader.cc:

699704

**Evaluate the reduction in size of the model.**

In [ ]:
print("Float model in Mb:", os.path.getsize(tflite_model_file) / float(2**20))
print("Quantized aware (QAT) model in Mb:", os.path.getsize(tflite_quantaware_model_file) / float(2**20))
print("Compression ratio:", os.path.getsize(tflite_model_file)/os.path.getsize(tflite_quantaware_model_file))

Float model in Mb: 1.7407188415527344
Quantized aware (QAT) model in Mb: 0.6672897338867188
Compression ratio: 2.6086402250094327


### Part (d): Evaluate the TF Lite QAT model accuracy
Hint: Use the intrepreter evaluate_model() function to get the accuracy result.

In [ ]:
import numpy as np

def evaluate_model(interpreter):
  input_index = interpreter.get_input_details()[0]["index"]
  output_index = interpreter.get_output_details()[0]["index"]

  # Run predictions on every image in the "test" dataset.
  prediction_fashion_type = []
  for i, test_image in enumerate(x_test):
    if i % 1000 == 0:
      print('Evaluated on {n} results so far.'.format(n=i))
    # Pre-processing: add batch dimension and convert to float32 to match with
    # the model's input data format.
    test_image = np.expand_dims(test_image, axis=0).astype(np.float32)
    interpreter.set_tensor(input_index, test_image)

    # Run inference.
    interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
    output = interpreter.tensor(output_index)
    fashion_type = np.argmax(output()[0])
    prediction_fashion_type.append(fashion_type)

  print('\n')
  # Compare prediction results with ground truth labels to calculate accuracy.
  prediction_fashion_type = np.array(prediction_fashion_type)
  accuracy = (prediction_fashion_type == y_test).mean()
  return accuracy

In [ ]:
interpreter = tf.lite.Interpreter(model_content=tflite_quantaware_model)
interpreter.allocate_tensors()

test_accuracy = evaluate_model(interpreter)

print('Quant TFLite test_accuracy:', test_accuracy)
print('Quant TF test accuracy:', q_aware_model_accuracy)

Evaluated on 0 results so far.
Evaluated on 1000 results so far.
Evaluated on 2000 results so far.
Evaluated on 3000 results so far.
Evaluated on 4000 results so far.
Evaluated on 5000 results so far.
Evaluated on 6000 results so far.
Evaluated on 7000 results so far.
Evaluated on 8000 results so far.
Evaluated on 9000 results so far.


Quant TFLite test_accuracy: 0.9134
Quant TF test accuracy: 0.9172000288963318


## <span style='color: red;'>Comment on the results:</span> ##


---

Through this exercise, I explored various quantization techniques and their effects on a convolutional neural network (CNN) trained on the Fashion MNIST dataset. Initially, I developed a baseline float32 model, which achieved a test accuracy of **90.60%** with a model size of approximately **1.74 MB**. This served as a reference point for evaluating how different quantization strategies affect both model size and accuracy.

I first applied **Float16 post-training quantization**, which reduced the model size by almost half to **0.87 MB** (a **1.99x compression ratio**) while retaining the same accuracy of **90.63%**. This demonstrated that converting to half precision has negligible impact on accuracy, yet offers a significant reduction in memory footprint.

Then, I explored **dynamic range quantization** (weights in int8 but float inputs/outputs). This approach shrank the model size further to **0.45 MB**, achieving a **3.89x compression ratio** with an accuracy of **90.64%**. Similarly, **full integer quantization** using uint8 for weights, inputs, and outputs, resulted in **0.45 MB size** and **90.59% accuracy**. Both methods confirmed that quantization can **significantly compress models** with **minimal accuracy trade-off**.

To maximize efficiency while preserving model performance, I implemented **Quantization-Aware Training (QAT)**. This method allowed the model to adapt to lower precision during training, and the results were impressive. The QAT-trained model achieved **91.72% accuracy**, even slightly exceeding the float32 baseline. When converted to TFLite, the quantized QAT model maintained **91.34% accuracy** and had a compressed size of **0.67 MB** (a **2.61x compression ratio**). Fine-tuning on just a **subset of training data** during QAT further enhanced accuracy, highlighting the method's adaptability.

---

### **Observations**
- **Accuracy Stability**: Across all quantization types, **accuracy remained above 90.5%**, validating that quantization has minimal impact on model performance, especially on balanced datasets like Fashion MNIST.
- **Compression vs. Performance**: **Float16 quantization** offered a moderate compression with **no accuracy loss**, while **full integer quantization** provided **maximum compression** with **slightly reduced accuracy**.
- **QAT Effectiveness**: Among all methods, **Quantization-Aware Training was the most effective**, achieving **near float32 accuracy** after quantization and offering a good balance between compression and performance.
- **Trade-off Understanding**: I observed that **post-training quantization is quicker and easier**, but QAT is more suitable when **accuracy retention is critical**, especially for models with tighter performance margins.

---

This exercise gave me valuable insights into the **practical trade-offs** between **model size, accuracy, and computational efficiency**. I learned how different quantization techniques serve different deployment needs, from lightweight models for microcontrollers to high-performance models for mobile devices. More importantly, I experienced how **Quantization-Aware Training can yield optimized models** without sacrificing accuracy, and how **fine-tuning even on smaller datasets can enhance performance**. Overall, this hands-on experience strengthened my understanding of how to prepare models for **real-world, resource-constrained environments** through quantization.